# How to use the data dependency checker for static array access

This is a small introduction notebook on the use of dependency detection inside of loop enviroments for static [<sup>(1)</sup>](#fn1)  array acesses. The intention is to show the required steps to setup the dependency problem, how to solve it and interpret the result.

Lets start by parsing the file `src/phys_mod.F90` from the `example` directory and let us consider the simple routine `phys_kernel_LITE_LOOP`.

[<sup id="fn1">(1)</sup>](#fn1-back) "static access is an array reference at a particular location in a programm" - Compilers: Principles, Techniques, and Tools </span>

In [1]:
from loki import Sourcefile

source = Sourcefile.from_file("src/phys_mod.F90", preprocess=True)
routine = source["phys_kernel_LITE_LOOP"]

[Loki::Sourcefile] Constructed from src/phys_mod.F90 in 1.64s


Let us examine that subroutine, it consists of a nested loop of depth two with two assignments in the inner most loop body. 

In [2]:
routine.body.view()

<Section::>
  <Loop:: k=1:dim2>
    <Loop:: i=i1:i2>
      <Assignment:: out1(i, k) = (in1(i, k) + in2(i, k) + in3(i, k) + in4(i, k) + in5(i, 
      k) + in6(i, k) + in7(i, k) + in8(i, k) + in9(i, k) + in10(i, k))*0.1>
      <Assignment:: in1(i, k) = out1(i, k)>

Before we consider the actual iteration space (i.e. space spanned by the loop indicies) we define some small utility functions. One simple interpreting anything as a column vector and one pretty printing a system of equations for better viewing experience.

In [3]:
from numpy import array, isscalar


def as_column(element):
    return array(element).reshape(-1, 1)


def pprint_equation(*args):
    def make_strings_equal_length(strings):
        max_length = max(len(s) for s in strings)
        padded_strings = [s.ljust(max_length) for s in strings]
        return padded_strings

    collection = [
        str(element).split("\n") if not isscalar(element) else str(element)
        for element in args
    ]
    count_rows_per_element = [
        len(element) if isinstance(element, list) else 1 for element in collection
    ]

    max_number_lines = max(count_rows_per_element)
    collection = [
        [" "] * ((max_number_lines - len(element) + 1) // 2)
        + element
        + [" "] * ((max_number_lines - len(element)) // 2)
        if isinstance(element, list)
        else element
        for index, element in enumerate(collection)
    ]

    collection = [
        make_strings_equal_length(element) if isinstance(element, list) else element
        for element in collection
    ]

    for i in range(max_number_lines):
        for element in collection:
            if isinstance(element, list):
                print(element[i], end=" ")
            else:
                if i == max_number_lines // 2:
                    print(element, end=" ")
                else:
                    print(" " * len(element), end=" ")
        print("")

Extracting the iteration space is an easy process. We know that all our assignments which are worth comparing are inside the deepest loop body therfore simply collecting all nested loops is enough and transforming them via the polyhedron utility into the required inequality relation. For a more complex situation view further down "Collecting access in different loop bodies"

In [4]:
from loki.analyse.analyse_dependency_detection import get_nested_loops
from loki.analyse.util_polyhedron import Polyhedron

nested_loops = list(get_nested_loops(routine.body))
poly = Polyhedron.from_nested_loops(nested_loops)
B, b = poly.A, as_column(poly.b)
iteration_space_required_variables = list(str(v) for v in poly.variables)

In [5]:
pprint_equation(B, "*", as_column(iteration_space_required_variables), "≤", b)

                     [['k']             
[[-1  0  0  0  0]     ['i']      [[-1]  
 [ 1  0 -1  0  0]  *  ['dim2'] ≤  [ 0]  
 [ 0 -1  0  1  0]     ['i1']      [ 0]  
 [ 0  1  0  0 -1]]    ['i2']]     [ 0]] 


Next is the collection of the actual array access that we would like to compare. With the knowledge of the subroutine we deal with only two assignments, by traversing the IR and collecting all assignments.

In [6]:
from loki import FindNodes, Assignment

assignments = FindNodes(Assignment).visit(routine.body)

first_assignement, second_assignment = (*assignments,)

print(
    f"Write into out1(i,k): \n\t{first_assignement}",
    f"Read of out1(i,k): \n\t{second_assignment}",
    sep="\n",
)

Write into out1(i,k): 
	Assignment:: out1(i, k) = (in1(i, k) + in2(i, k) + in3(i, k) + in4(i, k) + in5(i, k) + in6(i, k) + in7(i, k) + in8(i, k) + in9(i, k) + in10(i, k))*0.1
Read of out1(i,k): 
	Assignment:: in1(i, k) = out1(i, k)


The task of how to select which array access to compare with which is currently not automatized, therefore the user is required to produce a reasonable. Here we compare the write into `out1` of the first assignment with the read of it in the second assignment, which represents a **True Dependency**, i.e. a write which is folled by a read of the same location. Even though the overall relationship is more complex, since the read of `out1` is stored in `in1` which then may be used in the some future iteration.

Now we need to describe this access in their affine array access function representation, i.e. as a matrix vector product with an additional added vector. This is done by using the `construct_affine_array_access_function_representation` function which takes a tuple describing the dimensions access of an array and a list of to be considered variables.

In [7]:
from loki.analyse.analyse_dependency_detection import (
    construct_affine_array_access_function_representation,
)

F, f, access_variables = construct_affine_array_access_function_representation(
    first_assignement.lhs.dimensions, iteration_space_required_variables
)

Resulting in this representation:

In [8]:
pprint_equation(F, "*", as_column(access_variables), "+", f, "=", 0)

                [['k']                
                 ['i']                
[[0 1 0 0 0]  *  ['dim2'] + [[0]  = 0 
 [1 0 0 0 0]]    ['i1']      [0]]     
                 ['i2']]              


The same is done for the second access.

In [9]:
(
    F_dash,
    f_dash,
    access_variables_dash,
) = construct_affine_array_access_function_representation(
    second_assignment.rhs.dimensions, iteration_space_required_variables
)

assert access_variables == access_variables_dash

With the representation:

In [10]:
pprint_equation(F_dash, "*", as_column(access_variables_dash), "+", f_dash, "=", 0)

                [['k']                
                 ['i']                
[[0 1 0 0 0]  *  ['dim2'] + [[0]  = 0 
 [1 0 0 0 0]]    ['i1']      [0]]     
                 ['i2']]              


Now we have a iteration space per function access, here it is the same iteration space since all function accesses happen in the inner most loop. Passing this combination to the `has_data_dependency` function we now emply various techniques (gcd test, independet variable test, integer linear programming) to confirm or deny a data dependency.

In [11]:
from loki.analyse.analyse_array_data_dependency_detection import has_data_dependency

first_access_represenetation = ((B, b), (F, f))
second_access_represenetation = ((B, b), (F_dash, f_dash))

if has_data_dependency(first_access_represenetation, second_access_represenetation):
    print("There is a data dependency between the two accesses.")
else:
    print("There is no data dependency between the two accesses.")

There is a data dependency between the two accesses.


<a id='Collecting_access_in_different_loop_bodies'></a>
# Collecting access in different loop bodies

For this we consider a different subroutine, with two independent loops one iterating over all even integers and one iterating over all odd, clearly showing no data dependency.

In [12]:
source = Sourcefile.from_file(
    "../tests/sources/data_dependency_detection/loop_carried_dependencies.f90",
    preprocess=True,
)
routine = source["NoDependency"]

[Loki::Sourcefile] Constructed from ../tests/sources/data_dependency_detection/loop_carried_dependencies.f90 in 0.20s


In [13]:
routine.body.view()

<Section::>
  <Comment:: >
  <Loop:: i=1:10:1>
    <Assignment:: data(2*i) = 10>
  <Comment:: >
  <Loop:: i=1:5:1>
    <Assignment:: data(2*i + 1) = 20>

Here the two different loops are collected greedily and then **two** different iteration spaces are constructed in the same way as above.

In [14]:
from loki import Loop

outer_loops = FindNodes(Loop, greedy=True).visit(routine.body)

first_iteration_space = Polyhedron.from_nested_loops([outer_loops[0]])
second_iteration_space = Polyhedron.from_nested_loops([outer_loops[1]])

In [15]:
B, b = first_iteration_space.A, as_column(first_iteration_space.b)
iteration_space_required_variables = [str(v) for v in first_iteration_space.variables]

pprint_equation(B, "*", as_column(iteration_space_required_variables), "≤", b)

[[-1]              [[-1]  
 [ 1]] * [['i']] ≤  [10]] 


In [16]:
B_dash, b_dash = second_iteration_space.A, as_column(second_iteration_space.b)
iteration_space_required_variables_dash = [
    str(v) for v in second_iteration_space.variables
]

assert iteration_space_required_variables == iteration_space_required_variables_dash

pprint_equation(
    B_dash, "*", as_column(iteration_space_required_variables_dash), "≤", b_dash
)

[[-1]              [[-1]  
 [ 1]] * [['i']] ≤  [ 5]] 


Again the assignments are collected, then the access on the `data` array is represented for the two different assignments.

In [17]:
assignments = FindNodes(Assignment).visit(routine.body)

first_assignement, second_assignment = (*assignments,)

print(
    f"Write into data(even values): \n\t{first_assignement}",
    f"Write into data(odd values): \n\t{second_assignment}",
    sep="\n",
)

Write into data(even values): 
	Assignment:: data(2*i) = 10
Write into data(odd values): 
	Assignment:: data(2*i + 1) = 20


In [18]:
F, f, access_variables = construct_affine_array_access_function_representation(
    first_assignement.lhs.dimensions, iteration_space_required_variables
)

pprint_equation(F, "*", as_column(access_variables), "+", f, "=", 0)

[[2]] * [['i']] + [[0]] = 0 


In [19]:
(
    F_dash,
    f_dash,
    access_variables_dash,
) = construct_affine_array_access_function_representation(
    second_assignment.lhs.dimensions, iteration_space_required_variables
)

assert access_variables == access_variables_dash

pprint_equation(F_dash, "*", as_column(access_variables_dash), "+", f_dash, "=", 0)

[[2]] * [['i']] + [[1]] = 0 


And last but not least the question of a data dependency is answered.

In [20]:
first_access_represenetation = ((B, b), (F, f))
second_access_represenetation = ((B_dash, b_dash), (F_dash, f_dash))

if has_data_dependency(first_access_represenetation, second_access_represenetation):
    print("There is a data dependency between the two accesses.")
else:
    print("There is no data dependency between the two accesses.")

There is no data dependency between the two accesses.
